In [1]:
import json
import os
from moo_algorithm.metric import cal_hv
import numpy as np

def cal_HV_algorithm(data_name, number_customers, algorithm):
    data_path = os.path.join(
        "result",
        algorithm,
        f"N{number_customers}",
        f"{data_name}.json"
    )


    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    objectives_list = data["history"]["99"]
    hv_value = cal_hv(np.array(objectives_list), ref_point=np.array([100, 500000]))
    return hv_value


In [2]:
algorithms = ['NSGA_II', "NSGA_III" ,"MOEAD", "PFG_MOEA", "AGEA"]

for algor in algorithms:
    hv = cal_HV_algorithm("S042_N200_RC_R50", 200, algor)
    print(f"{algor}: HV = {hv}")    

NSGA_II: HV = 28096343.00511535
NSGA_III: HV = 29791248.726247035
MOEAD: HV = 22532071.727064934
PFG_MOEA: HV = 27775760.84514374
AGEA: HV = 28154751.765239436


In [6]:
import os
import json
import numpy as np
from pathlib import Path
from moo_algorithm.metric import cal_hv 

# --- Configuration ---
RESULT_DIR = "result"
# Since data is normalized to [0, 1], we use a standard 1.1 ref point
NORMALIZED_REF_POINT = np.array([1.1, 1.1]) 

def get_final_front(json_path):
    with open(json_path, 'r', encoding="utf-8") as f:
        data = json.load(f)
    history = data.get("history", {})
    if not history: return None
    last_gen = str(max([int(k) for k in history.keys()]))
    return np.array(history[last_gen])

def main():
    result_path = Path(RESULT_DIR)
    instance_groups = {}
    
    # 1. Group files by instance
    for json_file in result_path.glob("**/*.json"):
        if json_file.name != "S042_N100_R_R50.json":
            continue
        instance_name = json_file.name
        algorithm = json_file.parts[1] 
        if instance_name not in instance_groups:
            instance_groups[instance_name] = []
        instance_groups[instance_name].append((algorithm, json_file))

    print(f"{'Instance':<30} | {'Algorithm':<12} | {'Norm. HV (Ref 1.1)':<15}")
    print("-" * 70)

    for instance, entries in instance_groups.items():
        all_fronts_data = []
        
        # Load fronts
        for algo, path in entries:
            front = get_final_front(path)
            if front is not None:
                all_fronts_data.append((algo, front))

        if not all_fronts_data: continue

        # 2. Find Global Ideal and Nadir for this instance
        combined_points = np.vstack([data[1] for data in all_fronts_data])
        global_ideal = np.min(combined_points, axis=0)
        global_nadir = np.max(combined_points, axis=0)

        # Prevent division by zero if all solutions have the same objective value
        denom = global_nadir - global_ideal
        denom[denom == 0] = 1e-9 

        # 3. Normalize and calculate HV
        for algo, front in all_fronts_data:
            # Normalization Step
            normalized_front = (front - global_ideal) / denom
            
            # Calculate HV using the normalized front and the 1.1 reference
            hv_val = cal_hv(normalized_front, NORMALIZED_REF_POINT)
            
            print(f"{instance:<30} | {algo:<12} | {hv_val:>15.6f}")

if __name__ == "__main__":
    main()

Instance                       | Algorithm    | Norm. HV (Ref 1.1)
----------------------------------------------------------------------
S042_N100_R_R50.json           | NSGA_III     |        0.766679
S042_N100_R_R50.json           | NSGA_II      |        0.990223
S042_N100_R_R50.json           | IAGEA        |        1.079613
S042_N100_R_R50.json           | AGEA         |        0.935454
S042_N100_R_R50.json           | PFG_MOEA     |        0.498688
S042_N100_R_R50.json           | MOEAD        |        0.214445


In [12]:
import os
import json
import numpy as np
from pathlib import Path
from moo_algorithm.metric import cal_hv # Reusing your existing pymoo-based HV function

# --- Configuration ---
RESULT_DIR = "result"
# Since you are dividing by nadir, the nadir becomes [1, 1].
# We use 1.1 as the reference point to include the boundary solutions.
NORMALIZED_REF_POINT = np.array([1.1, 1.1]) 

def get_final_front(json_path):
    """Loads the last generation's objectives from the JSON file."""
    with open(json_path, 'r', encoding="utf-8") as f:
        data = json.load(f)
    
    history = data.get("history", {})
    if not history:
        return None
    
    # Identify the last generation (usually '99')
    last_gen = str(max([int(k) for k in history.keys()]))
    return np.array(history[last_gen])

def main():
    result_path = Path(RESULT_DIR)
    if not result_path.exists():
        print(f"Error: {RESULT_DIR} not found.")
        return

    # 1. Group files by instance to find the shared Nadir Point
    instance_groups = {}
    for json_file in result_path.glob("**/*.json"):
        if not json_file.match("S042_N100_*.json"):
            continue

        instance_name = json_file.name
        algorithm = json_file.parts[1] 
        
        if instance_name not in instance_groups:
            instance_groups[instance_name] = []
        instance_groups[instance_name].append((algorithm, json_file))

    print(f"{'Instance':<30} | {'Algorithm':<12} | {'Norm HV (v/Nadir)':<15}")
    print("-" * 70)

    for instance, entries in instance_groups.items():
        all_fronts_data = []
        
        # Load all algorithm results for this specific problem file
        for algo, path in entries:
            front = get_final_front(path)
            if front is not None:
                all_fronts_data.append((algo, front))

        if not all_fronts_data:
            continue

        # 2. Calculate the Global Nadir for this specific instance
        # This is the worst value seen by ANY algorithm on this problem
        combined_points = np.vstack([data[1] for data in all_fronts_data])
        global_nadir = np.max(combined_points, axis=0)
        
        # Avoid division by zero
        global_nadir[global_nadir == 0] = 1e-9

        # 3. Calculate HV for each algorithm using your normalization formula
        for algo, front in all_fronts_data:
            # Formula: normalized_objectives = objectives / nadir_point
            normalized_front = front / global_nadir
            
            # Calculate HV using normalized front and [1.1, 1.1]
            hv_val = cal_hv(normalized_front, NORMALIZED_REF_POINT)
            
            print(f"{instance:<30} | {algo:<12} | {hv_val:>15.6f}")
        
        print("\n")

if __name__ == "__main__":
    main()

Instance                       | Algorithm    | Norm HV (v/Nadir)
----------------------------------------------------------------------
S042_N100_C_R50.json           | NSGA_III     |        0.466901
S042_N100_C_R50.json           | NSGA_II      |        0.535070
S042_N100_C_R50.json           | IAGEA        |        0.536279
S042_N100_C_R50.json           | AGEA         |        0.604081
S042_N100_C_R50.json           | PFG_MOEA     |        0.513762
S042_N100_C_R50.json           | MOEAD        |        0.293955


S042_N100_RC_R50.json          | NSGA_III     |        0.470478
S042_N100_RC_R50.json          | NSGA_II      |        0.531350
S042_N100_RC_R50.json          | IAGEA        |        0.458969
S042_N100_RC_R50.json          | AGEA         |        0.456686
S042_N100_RC_R50.json          | PFG_MOEA     |        0.436781
S042_N100_RC_R50.json          | MOEAD        |        0.245973


S042_N100_R_R50.json           | NSGA_III     |        0.289598
S042_N100_R_R50.json       